# **TP DATA ANALYTIQUE**

## **📌 Introduction**
Les Jeux Olympiques sont l'une des compétitions sportives les plus prestigieuses, et la natation y occupe une place importante.  
L'objectif de cette étude est d'analyser les performances des pays et des nageurs de 1912 à 2020, en identifiant les nations et athlètes les plus médaillés ainsi que les évolutions des records olympiques.

Nous utiliserons le dataset **Olympic_Swimming_Results_1912to2020**, qui contient des informations détaillées sur les compétitions de natation olympiques.

Objectif du projet : analyser les performances des pays et des nageurs de 1912 à 2020.

📍 Location : Ville où se sont déroulés les JO.

📅 Year : Année de la compétition.

🏊 Distance (in meters) : Distance de l'épreuve en mètres.

🌊 Stroke : Type de nage (dos, brasse, papillon, nage libre, etc.).

🔄 Relay? : Indique si l’épreuve est un relais (1 = oui, 0 = non).

🚻 Gender : Genre des participants (hommes ou femmes).

🌎 Team : Pays ou comité représentant l'athlète.

🏅 Athlete : Nom de l’athlète.

⏱️ Results : Temps réalisé lors de l’épreuve.

🏆 Rank : Classement dans l’épreuve.

In [1]:
#importation des bibliothèques pour l'analyse de données
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
#importons le dataset et affichons quelques lignes de manière aléatoire
df = pd.read_csv('Olympic_Swimming_Results_1912to2020.csv')
df.sample(10)

,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Rank
3077,Munich,1972,400m,Individual medley,0,Men,SWE,Bengt Gingsjo,00:04:37.960000,4
1738,Atlanta,1996,50m,Freestyle,0,Men,BRA,Fernando Scherer,22.290,3
1589,Atlanta,1996,200m,Backstroke,0,Women,NZL,Anna Katrina Simcic,00:02:14.040000,4
2624,Moscow,1980,400m,Freestyle,0,Men,URS,Vladimir Salnikov,00:03:51.310000,1
1408,Sydney,2000,400m,Freestyle,0,Women,USA,Brooke Bennett,00:04:05.800000,1
1937,Barcelona,1992,400m,Individual medley,0,Women,CHN,Li Lin,00:04:36.730000,2
1781,Barcelona,1992,100m,Breaststroke,0,Men,EUN,Dmitri Volkov,00:01:02.070000,4
2964,Munich,1972,100m,Freestyle,0,Women,FRG,Heidemarie Reineck,59.730,4
494,London,2012,100m,Backstroke,0,Women,AUS,Belinda Hocking,59.290,4
2747,Montreal,1976,100m,Freestyle,0,Men,FRG,Klaus Steinbach,51.680,4


In [3]:
#Information sur notre dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4359 entries, 0 to 4358
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              4359 non-null   object
 1   Year                  4359 non-null   int64 
 2   Distance (in meters)  4359 non-null   object
 3   Stroke                4359 non-null   object
 4   Relay?                4359 non-null   int64 
 5   Gender                4359 non-null   object
 6   Team                  4359 non-null   object
 7   Athlete               4345 non-null   object
 8   Results               4331 non-null   object
 9   Rank                  4359 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 340.7+ KB


In [4]:
#description de notre dataset
#pour les variables quantitatives
A = df.describe()
#pour les variables qualitatives
B = df.describe(include='object')

print(A)
print()
print(B)

              Year       Relay?         Rank
count  4359.000000  4359.000000  4359.000000
mean   1982.936453     0.169764     3.164946
std      26.928344     0.375468     1.189715
min    1912.000000     0.000000     0.000000
25%    1968.000000     0.000000     2.000000
50%    1988.000000     0.000000     4.000000
75%    2004.000000     0.000000     4.000000
max    2020.000000     1.000000     5.000000

       Location Distance (in meters)     Stroke Gender  Team         Athlete  \
count      4359                 4359       4359   4359  4359            4345   
unique       22                    8          6      2    87            2821   
top       Tokyo                 200m  Freestyle    Men   USA  Michael Phelps   
freq        368                 1280       1973   2291   896              18   

              Results  
count            4331  
unique           3635  
top     Did not start  
freq               43  


### **Nettoyage du dataset**


In [5]:
def clean_time_string(time_str):
    """ Nettoie les valeurs en supprimant les caractères non numériques valides (sauf ':' et '.') """
    if isinstance(time_str, str):
        time_str = re.sub(r"[^0-9:.]", "", time_str)  # Garde uniquement les chiffres, ':' et '.'
    return time_str

def convert_to_seconds(time_str):
    """Convertit un temps au format hh:mm:ss ou mm:ss en secondes."""
    if isinstance(time_str, str) and ":" in time_str:
        parts = time_str.split(":")
        if len(parts) == 2:  # Format mm:ss
            minutes, seconds = map(float, parts)
            return minutes * 60 + seconds
        elif len(parts) == 3:  # Format hh:mm:ss
            hours, minutes, seconds = map(float, parts)
            return hours * 3600 + minutes * 60 + seconds
    try:
        return float(time_str)  # Si c'est déjà un nombre, on le garde tel quel
    except ValueError:
        return None  # Si la conversion échoue, on met None

# Étape 1 : Nettoyer les valeurs corrompues
df["Results"] = df["Results"].apply(clean_time_string)

# Étape 2 : Convertir en secondes
df["Results"] = df["Results"].apply(convert_to_seconds)

# Étape 3 : Supprimer les valeurs non convertibles
df = df.dropna(subset=["Results"])

In [6]:
print(df["Results"].dtype)

float64


In [7]:
# Vérifier le résultat
df.sample(5)

,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Rank
2996,Munich,1972,200m,Breaststroke,0,Men,MEX,Felipe Munoz,146.44,4
141,Tokyo,2020,100m,Freestyle,0,Women,NED,Femke Heemskerk,52.79,4
2563,Moscow,1980,200m,Backstroke,0,Men,URS,Vladimir Shemetov,123.48,4
2147,Seoul,1988,200m,Individual medley,0,Men,URS,Mikhail Zubkov,122.92,4
433,Rio,2016,4x100,Medley,1,Women,AUS,"Madison Wilson, Emily Seebohm, Taylor Mckeown,...",235.00,2


In [8]:
print(df['Year'].dtype)
print(df['Results'].dtype)
print(df['Relay?'].dtype)

int64
float64
int64


In [9]:
df['Relay?'].unique()

array([0, 1])

In [10]:
# Convertir la colonne 'Year' en entier
df['Year'] = df['Year'].astype(int)
df['Relay?'] = df['Relay?'].astype(bool)

<ipython-input-10-fb26f2ac68c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['Year'].astype(int)
<ipython-input-10-fb26f2ac68c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Relay?'] = df['Relay?'].astype(bool)


In [11]:
df['Team'].unique()

array(['ROC', 'USA', 'ITA', 'CHN', 'ESP', 'AUS', 'ROU', 'GBR', 'NED',
       'BLR', 'HUN', 'SUI', 'POL', 'GUA', 'BUL', 'FRA', 'KOR', 'UKR',
       'GER', 'AUT', 'JPN', 'FIN', 'SWE', 'RSA', 'BRA', 'LTU', 'NZL',
       'TUN', 'GRE', 'CAN', 'ISR', 'IRL', 'HKG', 'BEL', 'CZE', 'DEN',
       'RUS', 'KAZ', 'ISL', 'JAM', 'SGP', 'NOR', 'ZIM', 'NZ', 'NL', 'NLD',
       'SRB', 'CUB', 'ZAF', 'TTO', 'BAH', 'VEN', 'KEN', 'PNG', 'SLO',
       'CRO', 'SVK', 'ALG', 'ARG', 'CRC', 'PUR', 'MDA', 'BAR', 'EUN',
       'SUR', 'Unified Team ', 'TCH', 'URS', 'GDR', 'FRG', 'YUG', 'SUN',
       'BGR', 'COL', 'POR', 'CHE', 'MEX', 'ECU', 'PER', 'URU', 'EGY',
       'PHI', 'LUX', 'ANZ'], dtype=object)

In [12]:
#Remplaçons unified team par UFT si y a pas un autre UFT dedans
if "UFT" not in df['Team'].unique():
    df['Team'] = df['Team'].replace('Unified Team ', 'UFT')

df['Team'].unique()

<ipython-input-12-208e220cd512>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Team'] = df['Team'].replace('Unified Team ', 'UFT')


array(['ROC', 'USA', 'ITA', 'CHN', 'ESP', 'AUS', 'ROU', 'GBR', 'NED',
       'BLR', 'HUN', 'SUI', 'POL', 'GUA', 'BUL', 'FRA', 'KOR', 'UKR',
       'GER', 'AUT', 'JPN', 'FIN', 'SWE', 'RSA', 'BRA', 'LTU', 'NZL',
       'TUN', 'GRE', 'CAN', 'ISR', 'IRL', 'HKG', 'BEL', 'CZE', 'DEN',
       'RUS', 'KAZ', 'ISL', 'JAM', 'SGP', 'NOR', 'ZIM', 'NZ', 'NL', 'NLD',
       'SRB', 'CUB', 'ZAF', 'TTO', 'BAH', 'VEN', 'KEN', 'PNG', 'SLO',
       'CRO', 'SVK', 'ALG', 'ARG', 'CRC', 'PUR', 'MDA', 'BAR', 'EUN',
       'SUR', 'UFT', 'TCH', 'URS', 'GDR', 'FRG', 'YUG', 'SUN', 'BGR',
       'COL', 'POR', 'CHE', 'MEX', 'ECU', 'PER', 'URU', 'EGY', 'PHI',
       'LUX', 'ANZ'], dtype=object)

In [13]:
df['Stroke'].unique()

array(['Backstroke', 'Breaststroke', 'Butterfly', 'Freestyle',
       'Individual medley', 'Medley'], dtype=object)

In [14]:
#creation de la colonne medal
print(df['Rank'].unique())
#le nombre de quatrième place
print(df[df['Rank'] == 4].shape[0])

def create_medal(rank):
    if rank == 1:
        return 'Gold'
    elif rank == 2:
        return 'Silver'
    elif rank == 3:
        return 'Bronze'
    else:
        return 'No Medal'

df['Medal'] = df['Rank'].apply(create_medal)
#supprimons la colonne rank
df.drop(columns=['Rank'], inplace=True)
df.sample(5)

[1 2 3 4]
2582


<ipython-input-14-b13dd112d9b2>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Medal'] = df['Rank'].apply(create_medal)
<ipython-input-14-b13dd112d9b2>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Rank'], inplace=True)


,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Medal
1112,Athens,2004,200m,Freestyle,False,Men,AUS,Ian Thorpe,104.71,Gold
215,Tokyo,2020,50m,Freestyle,False,Women,USA,Abbey Weitzeil,24.41,No Medal
3850,London,1948,100m,Freestyle,False,Women,NED,Marie-Louise Jean Linssen-Vaessen,67.60,Bronze
625,London,2012,200m,Individual medley,False,Women,AUS,Alicia Coutts,128.15,Silver
3886,London,1948,400m,Freestyle,False,Men,GBR,Jack Irwin Hale,295.90,No Medal


In [15]:
#les valeurs manquantes
df.isnull().sum()

,0
Location,0
Year,0
Distance (in meters),0
Stroke,0
Relay?,0
Gender,0
Team,0
Athlete,14
Results,0
Medal,0


In [16]:
#supprimons les valeurs manquantes
df.dropna(inplace=True)
df.isnull().sum()

,0
Location,0
Year,0
Distance (in meters),0
Stroke,0
Relay?,0
Gender,0
Team,0
Athlete,0
Results,0
Medal,0


In [19]:
df['Distance (in meters)'].unique()

array(['100m', '1500m', '200m', '400m', '50m', '800m', '4x100', '4x200'],
      dtype=object)

In [20]:
# Remplacer les valeurs pour ajouter "m" si nécessaire
df["Distance (in meters)"] = df["Distance (in meters)"].replace({"4x100": "400m", "4x200": "800m"})

# Vérifier les valeurs uniques après correction
print(df["Distance (in meters)"].unique())

['100m' '1500m' '200m' '400m' '50m' '800m']


In [21]:
df.sample(10)

,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Medal
3318,City,1968,400m,Individual medley,False,Women,NZL,Kathleen Tui Shipston,334.60,No Medal
3710,Melbourne,1956,400m,Freestyle,False,Men,JPN,Koji Nonoshita,278.20,No Medal
1136,Athens,2004,200m,Individual medley,False,Women,UKR,Yana Klochkova,131.14,Gold
1675,Atlanta,1996,400m,Individual medley,False,Men,AUS,Matthew Stephen Dunn,256.66,No Medal
2300,Angeles,1984,100m,Butterfly,False,Women,CAN,Michelle Mac Pherson,61.58,No Medal
1375,Sydney,2000,200m,Freestyle,False,Men,AUS,Grant Hackett,109.46,No Medal
3735,Melbourne,1956,800m,Freestyle,True,Men,ZAF,"Peter John Duncan, Dennis George Ford",529.50,No Medal
3777,Helsinki,1952,200m,Breaststroke,False,Men,USA,Bowen Dow Stassforth,154.70,Silver
2787,Montreal,1976,200m,Breaststroke,False,Men,CAN,Graham Smith,139.42,No Medal
2069,Seoul,1988,100m,Freestyle,False,Women,NED,Karin Brienesse,56.15,No Medal


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4216 entries, 0 to 4358
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Location              4216 non-null   object 
 1   Year                  4216 non-null   int64  
 2   Distance (in meters)  4216 non-null   object 
 3   Stroke                4216 non-null   object 
 4   Relay?                4216 non-null   bool   
 5   Gender                4216 non-null   object 
 6   Team                  4216 non-null   object 
 7   Athlete               4216 non-null   object 
 8   Results               4216 non-null   float64
 9   Medal                 4216 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 333.5+ KB


In [27]:
#enregistrons le dataset
df.to_csv('Olympic_Swimming_Results_1912to2020_cleaned.csv', index=False)


In [26]:
#créons un autre dataset uniquement basé sur le team et les médailles
# Filtrer les médailles par type
#gold_medals = df[df["Medal"] == "Gold"].groupby("Team")["Medal"].count()
#silver_medals = df[df["Medal"] == "Silver"].groupby("Team")["Medal"].count()
#bronze_medals = df[df["Medal"] == "Bronze"].groupby("Team")["Medal"].count()

# Fusionner les résultats dans un DataFrame
#medal_summary = pd.DataFrame({
#    "Or": gold_medals,
#    "Argent": silver_medals,
#    "Bronze": bronze_medals
#}).fillna(0).astype(int)  # Remplacer NaN par 0 et convertir en entier

# Ajouter une colonne total
#medal_summary["Total"] = medal_summary.sum(axis=1)